In [19]:
# imports
import pandas as pd
import numpy as np
import tensorflow as tf

In [20]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [21]:
VOCAB_SIZE = 20000
MAX_LENGTH = 35

In [22]:
def load_data(csv_path):
    """Load dataset from CSV file"""
    df = pd.read_csv(csv_path)
    return df["premise"].astype(str).tolist(), df["hypothesis"].astype(str).tolist(), df["label"].values

In [23]:
val_data_path = input("Input the path to validation data: ")
val_premises, val_hypotheses, val_labels = load_data(val_data_path)

Input the path to validation data:  /kaggle/input/nlu-dataset/dev.csv


In [24]:
import pickle

tokenizer = input("Input the path to tokenizer: ")
with open(tokenizer, "rb") as f:
    tokenizer = pickle.load(f)

Input the path to tokenizer:  /kaggle/input/testing/tokenizer.pkl


In [25]:
def tokenize_and_pad(texts, tokenizer, max_length):
    """Convert text to sequences and pad"""
    sequences = tokenizer.texts_to_sequences(texts)
    return pad_sequences(sequences, maxlen=max_length, padding="post", truncating="post")

In [26]:
X_val_p = tokenize_and_pad(val_premises, tokenizer, MAX_LENGTH)
X_val_h = tokenize_and_pad(val_hypotheses, tokenizer, MAX_LENGTH)
y_val = to_categorical(val_labels, num_classes=2)

In [27]:
EMBEDDING_DIM = 300

In [28]:
def load_glove_embeddings(glove_file, embedding_dim):
    """
    Load the embeddings for every word in the file
    """
    embeddings_index = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

In [29]:
glove_path = input("Input the path to glove embeddings file: ")
embeddings_index = load_glove_embeddings(glove_path, EMBEDDING_DIM)

Input the path to glove embeddings file:  /kaggle/input/glove300/glove.6B.300d.txt


In [30]:
def create_embedding_matrix(embedding_index, word_index, embedding_dim, vocab_size):
    """
    Creates an embedding matrix from the GloVe embeddings.
    """
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    
    for word, index in word_index.items():
        if index >= vocab_size:
            break
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
    
    return embedding_matrix

In [31]:
word_index = tokenizer.word_index
embedding_matrix = create_embedding_matrix(embeddings_index, word_index, EMBEDDING_DIM, VOCAB_SIZE)

In [32]:
def soft_attention(premise, hypothesis):
    attention = Dot(axes=-1)([premise, hypothesis])
    premise_attn = Softmax(axis=-1)(attention)
    hypothesis_attn = Softmax(axis=-2)(attention)

    premise_aligned = Dot(axes=1)([premise_attn, hypothesis])
    hypothesis_aligned = Dot(axes=1)([hypothesis_attn, premise])
    return premise_aligned, hypothesis_aligned

In [33]:
lstm_path = input("Input the path to LSTM model: ")
bilstm_path = input("Input the path to BiLSTM model: ")
gru_path = input("Input the path to GRU model: ")
bigru_path = input("Input the path to BiGRU model: ")

Input the path to LSTM model:  /kaggle/input/ensemble/tensorflow2/default/1/nli_LSTM_model.keras
Input the path to BiLSTM model:  /kaggle/input/ensemble/tensorflow2/default/1/nli_BiLSTM_model.keras
Input the path to GRU model:  /kaggle/input/ensemble/tensorflow2/default/1/nli_GRU_model.keras
Input the path to BiGRU model:  /kaggle/input/ensemble/tensorflow2/default/1/nli_BiGRU_model.keras


In [34]:
from tensorflow.keras.models import load_model

In [35]:
models = {
    "LSTM": load_model(
        lstm_path,
        custom_objects={"soft_attention": soft_attention}
    ),
    "BiLSTM": load_model(
        bilstm_path,
        custom_objects={"soft_attention": soft_attention}
    ),
    "GRU": load_model(
        gru_path,
        custom_objects={"soft_attention": soft_attention}
    ),
    "BiGRU": load_model(
        bigru_path,
        custom_objects={"soft_attention": soft_attention}
    ),
}

In [36]:
import json

weights_path = input("Input the path to model accuracy weights: ")
with open(weights_path, "r") as f:
    weights = json.load(f)

Input the path to model accuracy weights:  /kaggle/input/ensemble-model-weights/model_accuracy_weights.json


In [37]:
weighted_sum = np.zeros((X_val_p.shape[0], 2), dtype=np.float32)
    
for name, model in models.items():
    raw_preds = model.predict([X_val_p, X_val_h])

    # Apply weighting and accumulate
    weighted_preds = raw_preds * weights[name]
    weighted_sum += weighted_preds

# Calculate final predictions
y_pred = np.argmax(weighted_sum, axis=1)

211/211 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step
211/211 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step
211/211 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step
211/211 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step


In [38]:
from sklearn.metrics import classification_report
print(classification_report(val_labels, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.7185    0.6636    0.6900      3258
           1     0.7059    0.7565    0.7303      3478

    accuracy                         0.7115      6736
   macro avg     0.7122    0.7100    0.7101      6736
weighted avg     0.7120    0.7115    0.7108      6736



In [39]:
def save_predictions_only(y_pred, output_path):
    """Save only the predictions to a CSV file with a 'predictions' column."""
    df = pd.DataFrame({"predictions": y_pred.flatten()})
    df.to_csv(output_path, index=False)
    print(f"Predictions saved to {output_path}")

In [40]:
save_predictions_only(y_pred, "Group_47_B_evaluation.csv.predict")

Predictions saved to Group_47_B_evaluation.csv.predict
